# Retrieval Augmented Generation (RAG)
<h3>For Question Answering over PDF Documents</h3>

This notebook demonstrates a vanilla implementation of a RAG pipeline for Question Answering (single-turn) over PDF documents. We start by ingesting the documents and indexing them using an open-source embedding model like `bge-small-en-v1.5` and a vector database like `chromadb`. We then move on to the two main components of the RAG pipeline: Retrieval and Generation. For retrieval, we use the same `bge-small-en-v1.5` model to retrieve the top-k relevant documents for a given query. For generation, we use an open-source LLM like `llama3-70b-8192` from Groq to generate the answer from the retrieved documents.

In [1]:
import chromadb
from llama_index.core import Document, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

/home/bhavesh/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Ingestion

In [2]:
reader = SimpleDirectoryReader(input_dir="documents", required_exts=[".pdf"], filename_as_id=True, file_metadata=None)
documents = reader.load_data()

In [3]:
def clean_metadata(document: Document) -> Document:
    document.metadata = {
        "page_label": document.metadata["page_label"],
        "file_name": document.metadata["file_name"]
    }
    return document

In [4]:
documents = list(map(clean_metadata, documents))

In [12]:
splitter = SentenceSplitter(
    chunk_size=500,
    chunk_overlap=100,
    include_metadata=True,
)

In [13]:
# Ex. Splitting the first document
splitter.get_nodes_from_documents([documents[0]])

[TextNode(id_='519eed3b-15e7-4fd1-b5b2-a975734eba0d', embedding=None, metadata={'page_label': '1', 'file_name': 'BC_via_Search_VPT.pdf'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='/mnt/d/Fidelity/LEAP Gen AI Workshop/pdf-chatbot-leap-workshop/documents/BC_via_Search_VPT.pdf_part_0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'BC_via_Search_VPT.pdf'}, hash='28219c85d590931b65fcf84700dc2228bb0e4f65a5cd935f58ec7b9775639446'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='96c673df-d138-4d27-8b79-c9ba35519145', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='dbc651dffb9d109abd81ea6a4de47b4281354da5c1bf3289da596491d5f2d125')}, text='Behavioral Cloning via S

In [6]:
# Load an open source embedding model
embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5", device='cuda')

In [11]:
# Ex. Embedding some nodes
embed_model(splitter.get_nodes_from_documents([documents[0]]))

[TextNode(id_='126964cc-7055-4c6e-b79b-d9bda8825927', embedding=[-0.05667181313037872, 0.05341920629143715, -0.021931292489171028, 0.004545822739601135, -0.027262045070528984, 0.028728431090712547, 0.01862359791994095, -0.0016266194870695472, 0.010081356391310692, 0.026402493938803673, 0.0513732023537159, -0.07471895962953568, -0.021772367879748344, 0.053887516260147095, 0.030736912041902542, 0.045956775546073914, -0.049757830798625946, 0.05065251514315605, 0.047887712717056274, -0.01884228177368641, -0.007115844637155533, -0.049016956239938736, 0.03415176272392273, -0.052417825907468796, -0.021597567945718765, 0.006594723090529442, -0.04067232459783554, -0.049976807087659836, 0.021770469844341278, -0.21818138659000397, 0.04820670560002327, -0.011911135166883469, 0.038087014108896255, -0.007789707742631435, 0.0052032507956027985, 0.019187867641448975, -0.038507286459207535, 0.03414782136678696, -0.06089456006884575, 0.027183813974261284, 0.02287820540368557, 0.031756315380334854, 0.016

In [7]:
# Creating a Chroma VectorDB instance
db = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db.get_or_create_collection("rag")

# Create a Chroma VectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a Simple Document Store as well
docstore = SimpleDocumentStore()

In [8]:
# Build the Embedding Pipeline
pipeline = IngestionPipeline(
    transformations=[
        splitter,
        embed_model,
    ],
    vector_store=vector_store,
    docstore=docstore,
)

_nodes = pipeline.run(documents=documents, show_progress=True)

Parsing nodes:   0%|          | 0/86 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 307/307 [00:23<00:00, 12.90it/s]


In [14]:
# Create vector index for retrieval
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

## Retrieval

In [15]:
retriever = index.as_retriever(similarity_top_k=3)

In [16]:
query_str = "Explain behavioral cloning via search"

In [17]:
retrieved_nodes = retriever.retrieve(query_str)
retrieved_nodes

[NodeWithScore(node=TextNode(id_='212b8134-5d22-42ab-93ca-4f4fc1634785', embedding=None, metadata={'page_label': '1', 'file_name': 'BC_via_Search_VPT.pdf'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='/mnt/d/Fidelity/LEAP Gen AI Workshop/pdf-chatbot-leap-workshop/documents/BC_via_Search_VPT.pdf_part_0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'BC_via_Search_VPT.pdf'}, hash='28219c85d590931b65fcf84700dc2228bb0e4f65a5cd935f58ec7b9775639446'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f7a45fbc-31b9-4936-aef6-626a484fecc0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='dbc651dffb9d109abd81ea6a4de47b4281354da5c1bf3289da596491d5f2d125')}, text='Behav

In [18]:
print(retrieved_nodes[0].node.get_content(metadata_mode='all'))

page_label: 1
file_name: BC_via_Search_VPT.pdf

Behavioral Cloning via Search in Video PreTraining Latent Space
Federico Malato*
University of Eastern Finland
fmalato@uef.ﬁFlorian Leopold*
University of Bielefeld
ﬂeopold@techfak.uni-bielefeld.deAmogh Raut
Indian Institute of Technology BHU
Ville Hautam ¨aki
University of Eastern FinlandAndrew Melnik
University of Bielefeld
Abstract
Our aim is to build autonomous agents that can solve
tasks in environments like Minecraft. To do so, we used
an imitation learning-based approach. We formulate our
control problem as a search problem over a dataset of
experts’ demonstrations, where the agent copies actions
from a similar demonstration trajectory of image-action
pairs. We perform a proximity search over the BASALT
MineRL-dataset in the latent representation of a Video
PreTraining model. The agent copies the actions from
the expert trajectory as long as the distance between the
state representations of the agent and the selected ex-
pert traje

## Simple Response Generation

In [20]:
GROQ_API_KEY = "YOUR API KEY HERE"

llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [28]:
def build_context_prompt(retrieved_nodes, query_str):
    context_str = "\n--------------------\n".join([node.get_content(metadata_mode='all') for node in retrieved_nodes])
    prompt = f"Given the following context: \n{context_str}\n--------------------\n\nPlease answer the following query given the above context: \n{query_str}\n\n"
    return prompt

In [29]:
# Generate a context prompt string based on the retrieved nodes and the query
print(build_context_prompt(retrieved_nodes, query_str))

Given the following context: 
page_label: 1
file_name: BC_via_Search_VPT.pdf

Behavioral Cloning via Search in Video PreTraining Latent Space
Federico Malato*
University of Eastern Finland
fmalato@uef.ﬁFlorian Leopold*
University of Bielefeld
ﬂeopold@techfak.uni-bielefeld.deAmogh Raut
Indian Institute of Technology BHU
Ville Hautam ¨aki
University of Eastern FinlandAndrew Melnik
University of Bielefeld
Abstract
Our aim is to build autonomous agents that can solve
tasks in environments like Minecraft. To do so, we used
an imitation learning-based approach. We formulate our
control problem as a search problem over a dataset of
experts’ demonstrations, where the agent copies actions
from a similar demonstration trajectory of image-action
pairs. We perform a proximity search over the BASALT
MineRL-dataset in the latent representation of a Video
PreTraining model. The agent copies the actions from
the expert trajectory as long as the distance between the
state representations of the agent a

In [30]:
response = llm.complete(build_context_prompt(retrieved_nodes, query_str))

In [31]:
print(response.text)

Based on the provided context, Behavioral Cloning (BC) via Search is a method used to build autonomous agents that can solve tasks in environments like Minecraft. Here's a breakdown of the approach:

**Overview**

The goal is to build an agent that can mimic human-like behavior in a Minecraft environment by copying actions from expert demonstrations.

**Key Components**

1. **Expert Demonstrations**: A dataset of expert trajectories is provided, containing image-action pairs.
2. **Video PreTraining (VPT) Model**: A pre-trained model is used to encode the expert demonstrations into a latent space.
3. **Search**: The agent performs a proximity search in the latent space to find the most similar situation (i.e., a set of consecutive observation-action pairs) to the current state of the agent.

**How it Works**

1. The agent encodes the current situation using the VPT model.
2. The agent searches for the nearest embedding point in the dataset of situation points.
3. Once the reference situ

# RAG Chatbot 
<h3>For Conversational Question Answering over PDF Documents</h3>

Now that we have a basic RAG pipeline in place, we can extend it to a conversational setting. We can use the same retrieval and generation components as before, but we also need to keep track of the conversation but more importantly, we need to intelligently call the RAG pipeline based on the context. We will create an Agent with function calling capabilities to achieve this.

In [48]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent

In [150]:
# Optional: If you have an OpenAI key, you can run this cell, else skip it.
# from llama_index.llms.openai import OpenAI

# OPENAI_API_KEY = "YOUR API KEY HERE"
# llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

## Simple Agent Demo

In [151]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)


def useless_tool() -> int:
    """This is a uselss tool."""
    return "This is a uselss output."


useless_tool = FunctionTool.from_defaults(fn=useless_tool)

In [152]:
# Add a system message for tool usage
system_message = ChatMessage(
    role='system',
    content="""You are a simple chatbot that has access to some tools.
You have access to the `add` function which can be used to add up any numbers in the query.
You don't need to call this function always, but you can use it when you need to do some addition-like calculation based on the query.
Use only when absolutely necessary.
"""
)

In [153]:
# We will use the OpenAIAgent class to use function calling capabilities with our Groq LLM
agent = OpenAIAgent.from_tools(
    tools=[useless_tool, add_tool], 
    llm=llm, 
    chat_history=[system_message],
    temperature=0,
    verbose=True, 
)

In [154]:
response = agent.chat(
    "help me settle a bet between me and my friend. I say 20 + 50 is equal to 70 but my friend says 80. who is correct?", tool_choice="auto"
)
print(response.response)

Added user message to memory: help me settle a bet between me and my friend. I say 20 + 50 is equal to 70 but my friend says 80. who is correct?
=== Calling Function ===
Calling function: add with args: {"a":20,"b":50}
Got output: 70

You are correct! 20 + 50 equals 70.


In [155]:
response = agent.chat(
    "yay! I won", tool_choice="auto"
)
print(response.response)

Added user message to memory: yay! I won
Congratulations on winning the bet! If you have any more questions or need assistance with anything else, feel free to ask.


## RAG Chatbot

In [156]:
def retrieval_fn(query: str) -> str:
    """This function let's you semantically retrieve relevant context chunks from a given document based on a query.

    Arguments:
        query (str): The query to search for in the document. Based on the original user query, write a good search query
                     which is more logically sound to retrieve the relevant information from the document.
    
    Returns:
        str: A string containing the retrieved context chunks from the document.
    """
    retrieved_nodes = retriever.retrieve(query)
    context_str = "\n--------------------\n".join([node.get_content(metadata_mode='all') for node in retrieved_nodes])
    return "Contextual information to answer the query is below:\n" + context_str

retrieval_tool = FunctionTool.from_defaults(fn=retrieval_fn)

In [157]:
system_message = ChatMessage(
    role='system',
    content="""You are a Q&A chatbot designed to answer questions based on a set of AI research paper PDFs. 
Your responses should be solely based on the information retrieved from these documents. 

To answer a question, first check if the answer can be derived from the current chat history context. If it can't,
you need to call the retrieval_fn() function with the appropriate search query. 

The retrieval_fn() function should only be called once per turn, ensuring efficient use of resources. 
If the function doesn't return the necessary information to support your response, you should inform the user that 
you are unable to provide an answer.

Remember, every answer you provide must be backed by a document retrieved from the retrieval_fn() function. This 
ensures that all responses are based on credible and verifiable information.
"""
)

In [158]:
rag_agent = OpenAIAgent.from_tools(
    tools=[retrieval_tool],
    llm=llm,
    chat_history=[system_message],
    temperature=0.1,
    verbose=True
)

In [159]:
rag_agent.chat_history

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are a Q&A chatbot designed to answer questions based on a set of AI research paper PDFs. \nYour responses should be solely based on the information retrieved from these documents. \n\nTo answer a question, first check if the answer can be derived from the current chat history context. If it can't,\nyou need to call the retrieval_fn() function with the appropriate search query. \n\nThe retrieval_fn() function should only be called once per turn, ensuring efficient use of resources. \nIf the function doesn't return the necessary information to support your response, you should inform the user that \nyou are unable to provide an answer.\n\nRemember, every answer you provide must be backed by a document retrieved from the retrieval_fn() function. This \nensures that all responses are based on credible and verifiable information.\n", additional_kwargs={})]

In [160]:
response = rag_agent.chat(
    "Explain behavioral cloning via search", tool_choice="auto"
)
print(response.response)

Added user message to memory: Explain behavioral cloning via search
=== Calling Function ===
Calling function: retrieval_fn with args: {"query":"behavioral cloning"}
Got output: Contextual information to answer the query is below:
page_label: 1
file_name: BC_via_Search_VPT.pdf

Behavioral Cloning via Search in Video PreTraining Latent Space
Federico Malato*
University of Eastern Finland
fmalato@uef.ﬁFlorian Leopold*
University of Bielefeld
ﬂeopold@techfak.uni-bielefeld.deAmogh Raut
Indian Institute of Technology BHU
Ville Hautam ¨aki
University of Eastern FinlandAndrew Melnik
University of Bielefeld
Abstract
Our aim is to build autonomous agents that can solve
tasks in environments like Minecraft. To do so, we used
an imitation learning-based approach. We formulate our
control problem as a search problem over a dataset of
experts’ demonstrations, where the agent copies actions
from a similar demonstration trajectory of image-action
pairs. We perform a proximity search over the BASALT
M

In [161]:
response = rag_agent.chat(
    "ELI5 your answer"
)
print(response.response)

Added user message to memory: ELI5 your answer
Behavioral cloning via search is like teaching a computer to imitate an expert by looking at how the expert solves problems. The computer learns by studying examples of the expert's actions and then tries to mimic those actions in similar situations. It uses a special model to understand and remember how the expert behaves optimally. By searching for similar situations in a dataset and copying the expert's actions, the computer can learn to act like the expert and perform tasks in a human-like way.
